In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME"], 1)
# application_df.head(5)

# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df.APPLICATION_TYPE.value_counts()

# Determine which values to replace if counts are less than 10,000?
replace_application = list(app_counts[app_counts <10000].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

class_counts = application_df.CLASSIFICATION.value_counts()

# Determine which values to replace if counts are less than 3000?
replace_class = list(class_counts[class_counts < 3000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    

# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T3,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [3]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)

# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [4]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 25

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               3500      
_________________________________________________________________
dense_1 (Dense)              (None, 25)                2525      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 26        
Total params: 6,051
Trainable params: 6,051
Non-trainable params: 0
_________________________________________________________________


In [5]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("optimize1_checkpoints/",exist_ok=True)
checkpoint_path = "optimize1_checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4020)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 815us/step - loss: 0.6099 - accuracy: 0.6893
Epoch 2/100
804/804 [==============================] - 1s 846us/step - loss: 0.5810 - accuracy: 0.7130
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5799 - accuracy: 0.7149
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5785 - accuracy: 0.7115
Epoch 5/100
783/804 [============================>.] - ETA: 0s - loss: 0.5750 - accuracy: 0.7140
Epoch 00005: saving model to optimize1_checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5751 - accuracy: 0.7140
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5814 - accuracy: 0.7086
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5722 - accuracy: 0.7178
Epoch 8/100
804/804 [==============================] - 1s 936us/step - loss: 0.5768 - accuracy: 0.7146
Epoch 9/100
804/804 [=============

804/804 [==============================] - 1s 733us/step - loss: 0.5676 - accuracy: 0.7209
Epoch 63/100
804/804 [==============================] - 1s 754us/step - loss: 0.5699 - accuracy: 0.7181
Epoch 64/100
804/804 [==============================] - 1s 691us/step - loss: 0.5666 - accuracy: 0.7207
Epoch 65/100
738/804 [==========================>...] - ETA: 0s - loss: 0.5731 - accuracy: 0.7165
Epoch 00065: saving model to optimize1_checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 700us/step - loss: 0.5728 - accuracy: 0.7167
Epoch 66/100
804/804 [==============================] - 1s 912us/step - loss: 0.5639 - accuracy: 0.7231
Epoch 67/100
804/804 [==============================] - 1s 722us/step - loss: 0.5686 - accuracy: 0.7179
Epoch 68/100
804/804 [==============================] - 1s 764us/step - loss: 0.5679 - accuracy: 0.7215
Epoch 69/100
804/804 [==============================] - 1s 691us/step - loss: 0.5664 - accuracy: 0.7194
Epoch 70/100
743/804 [=======

In [6]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5863 - accuracy: 0.7047
Loss: 0.5863499045372009, Accuracy: 0.704723060131073


In [7]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 25
hidden_nodes_layer3 = 20
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               3500      
_________________________________________________________________
dense_4 (Dense)              (None, 25)                2525      
_________________________________________________________________
dense_5 (Dense)              (None, 20)                520       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 21        
Total params: 6,566
Trainable params: 6,566
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Define the checkpoint path and filenames
os.makedirs("optimize1_checkpoints/",exist_ok=True)
checkpoint_path = "optimize1_checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4020)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 777us/step - loss: 0.6092 - accuracy: 0.6896
Epoch 2/100
804/804 [==============================] - 1s 828us/step - loss: 0.5790 - accuracy: 0.7138
Epoch 3/100
804/804 [==============================] - 1s 748us/step - loss: 0.5793 - accuracy: 0.7108
Epoch 4/100
804/804 [==============================] - 1s 840us/step - loss: 0.5797 - accuracy: 0.7119
Epoch 5/100
752/804 [===========================>..] - ETA: 0s - loss: 0.5790 - accuracy: 0.7117
Epoch 00005: saving model to optimize1_checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 758us/step - loss: 0.5790 - accuracy: 0.7117
Epoch 6/100
804/804 [==============================] - 1s 781us/step - loss: 0.5807 - accuracy: 0.7093
Epoch 7/100
804/804 [==============================] - 1s 765us/step - loss: 0.5779 - accuracy: 0.7125
Epoch 8/100
804/804 [==============================] - 1s 925us/step - loss: 0.5762 - accuracy: 0.7160
Epoch 9/100
804/804 [===

804/804 [==============================] - 1s 752us/step - loss: 0.5683 - accuracy: 0.7189
Epoch 63/100
804/804 [==============================] - 1s 822us/step - loss: 0.5683 - accuracy: 0.7197
Epoch 64/100
804/804 [==============================] - 1s 749us/step - loss: 0.5691 - accuracy: 0.7165
Epoch 65/100
780/804 [============================>.] - ETA: 0s - loss: 0.5707 - accuracy: 0.7179
Epoch 00065: saving model to optimize1_checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 859us/step - loss: 0.5707 - accuracy: 0.7179
Epoch 66/100
804/804 [==============================] - 1s 802us/step - loss: 0.5711 - accuracy: 0.7156
Epoch 67/100
804/804 [==============================] - 1s 749us/step - loss: 0.5708 - accuracy: 0.7192
Epoch 68/100
804/804 [==============================] - 1s 814us/step - loss: 0.5727 - accuracy: 0.7150
Epoch 69/100
804/804 [==============================] - 1s 733us/step - loss: 0.5685 - accuracy: 0.7187
Epoch 70/100
739/804 [=======

In [9]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5852 - accuracy: 0.7069
Loss: 0.5851911306381226, Accuracy: 0.7069388031959534


In [10]:
# Define the checkpoint path and filenames
os.makedirs("optimize2_checkpoints/",exist_ok=True)
checkpoint_path = "optimize2_checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4020)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 803us/step - loss: 0.5628 - accuracy: 0.7231
Epoch 2/100
804/804 [==============================] - 1s 759us/step - loss: 0.5633 - accuracy: 0.7249
Epoch 3/100
804/804 [==============================] - 1s 863us/step - loss: 0.5647 - accuracy: 0.7223
Epoch 4/100
804/804 [==============================] - 1s 756us/step - loss: 0.5651 - accuracy: 0.7225
Epoch 5/100
767/804 [===========================>..] - ETA: 0s - loss: 0.5662 - accuracy: 0.7189
Epoch 00005: saving model to optimize2_checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 811us/step - loss: 0.5662 - accuracy: 0.7189
Epoch 6/100
804/804 [==============================] - 1s 845us/step - loss: 0.5608 - accuracy: 0.7255
Epoch 7/100
804/804 [==============================] - 1s 752us/step - loss: 0.5717 - accuracy: 0.7160
Epoch 8/100
804/804 [==============================] - 1s 775us/step - loss: 0.5636 - accuracy: 0.7238
Epoch 9/100
804/804 [===

804/804 [==============================] - 1s 745us/step - loss: 0.5643 - accuracy: 0.7225
Epoch 63/100
804/804 [==============================] - 1s 744us/step - loss: 0.5721 - accuracy: 0.7139
Epoch 64/100
804/804 [==============================] - 1s 820us/step - loss: 0.5639 - accuracy: 0.7221
Epoch 65/100
778/804 [============================>.] - ETA: 0s - loss: 0.5640 - accuracy: 0.7223
Epoch 00065: saving model to optimize2_checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 798us/step - loss: 0.5641 - accuracy: 0.7222
Epoch 66/100
804/804 [==============================] - 1s 775us/step - loss: 0.5625 - accuracy: 0.7236
Epoch 67/100
804/804 [==============================] - 1s 806us/step - loss: 0.5698 - accuracy: 0.7155
Epoch 68/100
804/804 [==============================] - 1s 829us/step - loss: 0.5617 - accuracy: 0.7254
Epoch 69/100
804/804 [==============================] - 1s 743us/step - loss: 0.5693 - accuracy: 0.7161
Epoch 70/100
788/804 [=======

In [11]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5914 - accuracy: 0.7050
Loss: 0.5914089679718018, Accuracy: 0.7049562931060791


In [12]:

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  len(X_train[0]) * 4
hidden_nodes_layer2 = len(X_train[0]) *3
hidden_nodes_layer3 = len(X_train[0]) *2
hidden_nodes_layer4 = len(X_train[0])
hidden_nodes_layer4 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# fifth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 136)               4760      
_________________________________________________________________
dense_8 (Dense)              (None, 102)               13974     
_________________________________________________________________
dense_9 (Dense)              (None, 68)                7004      
_________________________________________________________________
dense_10 (Dense)             (None, 68)                4692      
_________________________________________________________________
dense_11 (Dense)             (None, 68)                4692      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 69        
Total params: 35,191
Trainable params: 35,191
Non-trainable params: 0
__________________________________________________

In [13]:
# Define the checkpoint path and filenames
os.makedirs("optimize3_checkpoints/",exist_ok=True)
checkpoint_path = "optimize3_checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", 
            optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam"), 
            metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4020)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6191 - accuracy: 0.6698
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5882 - accuracy: 0.7079
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5789 - accuracy: 0.7144
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5814 - accuracy: 0.7134
Epoch 5/100
776/804 [===========================>..] - ETA: 0s - loss: 0.5802 - accuracy: 0.7098
Epoch 00005: saving model to optimize3_checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5802 - accuracy: 0.7098
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5784 - accuracy: 0.7152
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5732 - accuracy: 0.7173
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5743 - accuracy: 0.7158
Epoch 9/100
804/804 [===================

804/804 [==============================] - 1s 1ms/step - loss: 0.5688 - accuracy: 0.7158
Epoch 64/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5658 - accuracy: 0.7222
Epoch 65/100
790/804 [============================>.] - ETA: 0s - loss: 0.5600 - accuracy: 0.7243
Epoch 00065: saving model to optimize3_checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5602 - accuracy: 0.7242
Epoch 66/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5715 - accuracy: 0.7154
Epoch 67/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5665 - accuracy: 0.7197
Epoch 68/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5680 - accuracy: 0.7201
Epoch 69/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5728 - accuracy: 0.7190
Epoch 70/100
768/804 [===========================>..] - ETA: 0s - loss: 0.5635 - accuracy: 0.7235
Epoch 00070: saving model to optimize3_checkpoint

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

268/268 - 0s - loss: 0.5860 - accuracy: 0.7040
Loss: 0.5860185027122498, Accuracy: 0.7040233016014099
